In [2]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import logging
from multiprocessing import Pool
import pandas as pd
import itertools

## Load Enviornment Variables

In [29]:
load_dotenv()
USERNAME = os.getenv('USERNAME_NEO4J')
PASS = os.getenv('PASSWORD_NEO4J')
URI = 'neo4j://localhost:7687'

## Configuring Logging

In [4]:
# Configure logging
logging.basicConfig(filename='neo4j_import.log',
                     level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


## Neo4j Driver Setup

In [ ]:
# Function to connect to Neo4j
def connect_to_neo4j(uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    return driver

driver = connect_to_neo4j(URI, USERNAME, PASS)

## Illicit Smart Contract Transactions

### Cypher Queries - Graph Schema

In [30]:


def import_data_from_csv(driver):
    with driver.session() as session:
        query = """
            LOAD CSV WITH HEADERS FROM 'file:///illicit_transactions/illicit_transactions_eth.csv' AS row
            WITH row SKIP 120500
            CALL {
                WITH row 
                MERGE (from:Account {address: COALESCE(row.from, 'Unknown')})
                MERGE (to:Account {address: COALESCE(row.to, 'Unknown')})

                MERGE (block:Block {blockNumber: toInteger(row.blockNumber), timeStamp: toInteger(row.timeStamp)})
                
                WITH row, toFloat(row.value) / 1.0E18 AS value
                
                MATCH (from:Account {address: COALESCE(row.from, 'Unknown')})
                MATCH (to:Account {address: COALESCE(row.to, 'Unknown')})
                CREATE (from)-[:TRANSFER {
                    timeStamp: toInteger(row.timeStamp),
                    hash: row.hash,
                    value: value,
                    gas: toInteger(row.gas),
                    gasPrice: toInteger(row.gasPrice),
                    isError: toBoolean(row.isError),
                    reputabilityFlag: toInteger(row.FLAG)
                }]->(to)

                WITH row
                MATCH (block:Block {blockNumber: toInteger(row.blockNumber), timeStamp: toInteger(row.timeStamp)})
                MATCH (from:Account {address: COALESCE(row.from, 'Unknown')})
                MATCH (to:Account {address: COALESCE(row.to, 'Unknown')})

                CREATE (block)-[:CONTAINS]->(from)
                CREATE (block)-[:CONTAINS]->(to)

                FOREACH (ignore IN CASE WHEN row.contractAddress IS NOT NULL THEN [1] ELSE [] END |
                    MERGE (sc:SmartContract {contractAddress: row.contractAddress})
                )

                With row
                MATCH (from:Account {address: COALESCE(row.from, 'Unknown')})
                MATCH (to:Account {address: COALESCE(row.to, 'Unknown')})
                OPTIONAL MATCH (sc:SmartContract {contractAddress: row.contractAddress})

                WITH from, to, sc
                WHERE sc IS NOT NULL

                FOREACH(ignore IN CASE WHEN sc IS NOT NULL THEN [1] ELSE [] END |
                    CREATE (from)-[:INVOKES]->(sc)
                    CREATE (to)-[:INVOKES]->(sc)
                )
            } IN TRANSACTIONS OF 500 ROWS
        """
        session.run(query) 

# Function to close the Neo4j connection
def close_neo4j_connection(driver):
    driver.close()

### Pre Analysis - Illicit

In [18]:
ill_df = pd.read_csv('../illicit_transaction_batches/illicit_transactions_eth.csv')


In [19]:
(ill_df.isna().sum() / len(ill_df)) * 100

blockNumber         0.000000
timeStamp           0.000000
hash                0.000000
from                0.000000
to                  0.059556
value               0.000000
gas                 0.000000
gasPrice            0.000000
isError             0.000000
contractAddress    99.940444
FLAG                0.000000
dtype: float64

### Pushing to Neo4j - Execution

In [25]:
driver = connect_to_neo4j(URI, USERNAME, PASS)
import_data_from_csv(driver)  # Set include_contracts to True if needed
close_neo4j_connection(driver)

Adding Reputability Flag to Account nodes

In [32]:
df = pd.read_csv('../fetch_illicit_accounts/illicit_addresses.csv')

with driver.session() as session:
    for index, row in df.iterrows():
        address = row['addresses.ETH']
        reputability_flag = 0
        
        query = (
            f"MATCH (account:Account {{address: '{address}'}}) "
            f"SET account.reputabilityFlag = 0"
        )
        
        session.run(query)

driver.close()

## Reputable Smart Contracts

### Cypher Queries 

In [33]:
def import_data_from_csv(driver, file_name):
    logging.info(f"Importing data from CSV file: {file_name}")
    with driver.session() as session:
        query = f"""
            LOAD CSV WITH HEADERS FROM 'file:///reputable_transactions/{file_name}.csv' AS row
            CALL {{
                WITH row 
                MERGE (from:Account {{address: COALESCE(row.from, 'Unknown')}})
                MERGE (to:Account {{address: COALESCE(row.to, 'Unknown')}})

                MERGE (block:Block {{blockNumber: toInteger(row.blockNumber), timeStamp: toInteger(row.timeStamp)}})
                
                WITH row, toFloat(row.value) / 1.0E18 AS value
                
                MATCH (from:Account {{address: COALESCE(row.from, 'Unknown')}})
                MATCH (to:Account {{address: COALESCE(row.to, 'Unknown')}})
                CREATE (from)-[:TRANSFER {{
                    timeStamp: toInteger(row.timeStamp),
                    hash: row.hash,
                    value: value,
                    gas: toInteger(row.gas),
                    gasPrice: toInteger(row.gasPrice),
                    isError: toBoolean(row.isError),
                    reputabilityFlag: 1
                }}]->(to)

                WITH row
                MATCH (block:Block {{blockNumber: toInteger(row.blockNumber), timeStamp: toInteger(row.timeStamp)}})
                MATCH (from:Account {{address: COALESCE(row.from, 'Unknown')}})
                MATCH (to:Account {{address: COALESCE(row.to, 'Unknown')}})

                CREATE (block)-[:CONTAINS]->(from)
                CREATE (block)-[:CONTAINS]->(to)

                FOREACH (ignore IN CASE WHEN row.contractAddress IS NOT NULL THEN [1] ELSE [] END |
                    MERGE (sc:SmartContract {{contractAddress: row.contractAddress}})
                )

                WITH row
                MATCH (from:Account {{address: COALESCE(row.from, 'Unknown')}})
                MATCH (to:Account {{address: COALESCE(row.to, 'Unknown')}})
                OPTIONAL MATCH (sc:SmartContract {{contractAddress: row.contractAddress}})

                WITH from, to, sc
                WHERE sc IS NOT NULL

                FOREACH(ignore IN CASE WHEN sc IS NOT NULL THEN [1] ELSE [] END |
                    CREATE (from)-[:INVOKES]->(sc)
                    CREATE (to)-[:INVOKES]->(sc)
                )
            }} IN TRANSACTIONS OF 500 ROWS
        """
        logging.info(f"Data import from CSV file {file_name} completed")
        session.run(query)

### Pushing to Neo4j - Execution

In [ ]:
def import_data_from_multiple_csv(driver):
    logging.info("Starting data import from multiple CSV files - Reputable Transactions")

    for i in range(20):  # 20 batch files of reputable transactions
        file_name = f"eth_trans_batch_{i}"
        import_data_from_csv(driver, file_name)

    logging.info("Data import from multiple CSV files completed - Reputable Transactions")

# Call the function to import data from all CSV files
import_data_from_multiple_csv(driver)